# Install pyspark

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=b9e1d1d3733adbd5b04363f6d11abdc141a0e34978db3bbf167a2024c8bec1b5
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
# Montage drive (optionnel)
from google.colab import drive
drive.mount('/My_drive/')

Mounted at /My_drive/


# TP numero 1

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
#Charger un csv (modifier le path)
path="/My_drive/My Drive/big data/Projet/offensive_comment_small.csv"
df = spark.read.csv(path,header=True, sep=";")
print(df.count())
df.show()

10000
+------------+--------------------+-----+------------+-------+------+------+-------------+
|          id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+------------+--------------------+-----+------------+-------+------+------+-------------+
| 86236166566|How can Ceha use ...|    0|           0|      0|     0|     0|            0|
| 67516955649|Blah... Childlove...|    0|           0|      0|     0|     0|            0|
| 22144619496|WTH DUDE Deleting...|    0|           0|      0|     0|     1|            0|
| 10428262239|"'""I''ve added a...|    0|           0|      0|     0|     0|            0|
| 82779776033|Violation Your re...|    0|           0|      0|     0|     0|            0|
| 72769334443|take care  16:55,...|    0|           0|      0|     0|     0|            0|
| 80250530594|'The second examp...|    0|           0|      0|     0|     0|            0|
| 40748094103|"'"" Bye I''m not...|    1|           0|      1|     0|     1|        

# Nettoyer la chaine de caractère

In [ ]:
# Dans un premier temps, faire une fonction qui fonctionne sur un exemple simple

import spacy

nlp = spacy.load("en_core_web_sm", disable=["ner"])

def clean_text(text):
  doc = nlp(text)
  tab_result = []
  for token in doc:
    if (token.is_stop == False) & (len(token.text) > 2) and token.pos_!="PUNCT":
      tab_result.append(token.lemma_)
  # Ne garder que les LEMME, enlever les STOPWORDS et les mots de moins de 2 charactères
  # A vous de coder
  return tab_result

# modifier la phrase 
clean_text("This IS a test, we are currently in a big data lesson and we try spacy")

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


['test', 'currently', 'big', 'datum', 'lesson', 'try', 'spacy']

In [ ]:
# A l'aide du premier TP et des UDF, appliquez votre fonction sur la colonne "comment_text"
df_test= df.withColumn('Nettoyage', F.udf(lambda a : clean_text(a),T.ArrayType(T.StringType()))("comment_text"))
df_test.show()

+------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
|          id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|           Nettoyage|
+------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+
| 86236166566|How can Ceha use ...|    0|           0|      0|     0|     0|            0|[Ceha, use, word,...|
| 67516955649|Blah... Childlove...|    0|           0|      0|     0|     0|            0|[Blah, Childlove,...|
| 22144619496|WTH DUDE Deleting...|    0|           0|      0|     0|     1|            0|[wth, dude, delet...|
| 10428262239|"'""I''ve added a...|    0|           0|      0|     0|     0|            0|[add, citation, E...|
| 82779776033|Violation Your re...|    0|           0|      0|     0|     0|            0|[violation, recen...|
| 72769334443|take care  16:55,...|    0|           0|      0|     0|     0|            0|[care, 16:55, 

## Sauvegarder dans un fichier parquet, recharger vos données

In [ ]:
df_test.write.parquet("/My_drive/My Drive/big data/Projet/clean.pqt")

In [ ]:
df_test = spark.read.parquet("/My_drive/My Drive/big data/Projet/clean.pqt")
df_test.count()

10000

Prochaine étapes :
- Fit, transform sur un algo de "vectorisation" CountVectorizer, TFIDF, W2V etc...
- Entrainer un modele sur la sortie de la "vectorisation"

Tips :
- Attention aux types, valeurs manquantes etc...
- Evaluer votre modèle à chaque tests
- Tester différentes approches !
- cf slides

In [ ]:
from pyspark.ml.feature import CountVectorizer

from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.mllib.evaluation import BinaryClassificationMetrics


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.classification import LogisticRegression

In [ ]:

cv = CountVectorizer(inputCol = "Nettoyage", outputCol="features",vocabSize = 2800, minDF = 1.0)
model = cv.fit(df_test)
result = model.transform(df_test)
result.show(truncate=False)


+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Pour la suite :
Vous êtes en autonomie, à vous d'experimenter différentes approches.
Chaque approche (même si elle ne donne pas de bon résultats) est intéressante à documenter.

Quelques pistes (Slides 63+64 du cours) :
- Mesurer les performances de vos modèles (et justifier le choix de votre score)
- Tenir un excel/autre doc à jour avec vos scores pour chaque experimentation
- Gerer les données déséquilibrer
- Tester un Word2Vec
- Ajouter un IDF après votre countVectorizer

In [ ]:
result.printSchema()
result= result.withColumn('toxic', result["toxic"].cast("float"))
result2= result.dropna()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: string (nullable = true)
 |-- severe_toxic: string (nullable = true)
 |-- obscene: string (nullable = true)
 |-- threat: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- identity_hate: string (nullable = true)
 |-- Nettoyage: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)



Train Test Split sur PySpark

In [ ]:
training_data, test_data = result2.randomSplit([0.8, 0.2], seed=1234)

In [ ]:
print(training_data.count())
print(test_data.count())

8020
1978


# RandomForestClassifier

In [ ]:
label = "toxic"
rf = RandomForestClassifier(numTrees=10, maxDepth=25,featuresCol='features', labelCol=label)
rf_model = rf.fit(training_data)
df_pred = rf_model.transform(test_data)
df_pred.show()

+------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+--------------------+----------+
|          id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|           Nettoyage|            features|       rawPrediction|         probability|prediction|
+------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+--------------------+----------+
| 10018353480|'Hey, you were ri...|  0.0|           0|      0|     0|     0|            0|[hey, right, thin...|(2800,[3,7,8,10,1...|[8.54959939581196...|[0.85495993958119...|       0.0|
|100311196363|user:captain-pois...|  1.0|           0|      0|     0|     0|            0|[user, captain, p...|(2800,[1,7,30,148...|[6.66456200797915...|[0.66645620079791...|       0.0|
|101451150796|Piss off you slan...|  1.0|           0|      1|     0| 

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="toxic", rawPredictionCol="prediction")
accuracy = evaluator.evaluate(df_pred)

print("Accuracy: %g" % (accuracy))

Accuracy: 0.678127


# Regression Logistique

In [ ]:
label = "toxic"
lr = LogisticRegression(featuresCol='features', labelCol=label)
lr_model = lr.fit(training_data)
df_pred = lr_model.transform(test_data)
df_pred.show()

+------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+--------------------+----------+
|          id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|           Nettoyage|            features|       rawPrediction|         probability|prediction|
+------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+--------------------+----------+
| 10018353480|'Hey, you were ri...|  0.0|           0|      0|     0|     0|            0|[hey, right, thin...|(2800,[3,7,8,10,1...|[92.8980410026275...|           [1.0,0.0]|       0.0|
|100311196363|user:captain-pois...|  1.0|           0|      0|     0|     0|            0|[user, captain, p...|(2800,[1,7,30,148...|[-70.537332532684...|[2.32287432060440...|       1.0|
|101451150796|Piss off you slan...|  1.0|           0|      1|     0| 

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="toxic", rawPredictionCol="prediction")
accuracy = evaluator.evaluate(df_pred)

print("Accuracy: %g" % (accuracy))

Accuracy: 0.788502
